In [2]:
import time
import numpy as np
from openmm import app
import openmm as mm
from openmm import unit as u
import pdbfixer
from sys import stdout

In [7]:
STRIDE = 100 #5000
simulations_steps = 10000 #50000000
temperature = 300 * u.kelvin
platform_name = 'CPU'#'OpenCL' 
real_time_STRIDE = 1000
friction_coeff = 1.0/u.picoseconds
time_step = 2.0*u.femtoseconds 
pressure = 1 * u.bar
maxSize=2*u.nanometer
geompadding = 3*u.nanometer

In [8]:
# Loading the file into the notebook.
pdb_file = 'data/peptide_fixed.pdb'
fixer = pdbfixer.PDBFixer(pdb_file)

In [9]:
vectors = mm.Vec3(1,0,0), mm.Vec3(1/3,2*np.sqrt(2)/3,0), mm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]

In [10]:
# Defining the forcfield and the model
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
model = app.Modeller(fixer.topology, fixer.positions)
model.addSolvent(forcefield,
             #padding=1.*u.nanometer,
             #boxSize= 'boxType',
                 boxVectors= boxVectors,#Box('truncatedOctahedron'),#mm.Vec3(3.0, 3.0, 3.0) * u.nanometers ,
             
                 model='tip3p', 
             ionicStrength=0.1 * u.millimolar, 
             positiveIon='Na+',
             negativeIon='Cl-',
             neutralize=True )

app.PDBFile.writeFile(model.topology,model.positions, open('data/peptide_model_water.pdb', 'w'))###add

system = forcefield.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1.2*u.nanometers, #we did 1.0
                                 constraints=app.HBonds, 
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

system.addForce(mm.MonteCarloBarostat(pressure, temperature))###add

integrator = mm.LangevinIntegrator(temperature,
                                   friction_coeff,
                                   time_step)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName(platform_name)
simulation = app.Simulation(model.topology, system, integrator, platform)
simulation.context.setPositions(model.positions)

#start = time.time()###delete
#simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data/output.pdb', STRIDE))###delete
simulation.reporters.append(mm.app.dcdreporter.DCDReporter('data/output.dcd', STRIDE))###add
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data/mm.log', STRIDE, step=True, potentialEnergy=True, temperature=True, time=True))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout, real_time_STRIDE, step=True, potentialEnergy=True, temperature=True, speed=True, elapsedTime=True))
simulation.minimizeEnergy()
simulation.step(simulations_steps)
#end = time.time()

#print(f"Elapsed real time: {end-start} s")###delete
#print(f'I could do {(60*60)/(end-start)*simulations_steps} simulation steps per hour with {simulations_steps/STRIDE} frames saved.')###delete

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-152690.46680971084,255.27975482713367,0,0.00015282630920410156
2000,-149151.82600934093,293.41081984538386,13.2,13.102509021759033
3000,-147945.9998107684,294.28973359974304,13,26.52563786506653
4000,-148802.6220300707,299.77743054414105,13,39.842286109924316
5000,-148248.25655550716,298.3756548403623,13,53.32757496833801
6000,-147643.70453877855,304.6883048017305,12.9,66.92929196357727
7000,-147701.61894552654,302.758216506575,12.9,80.4095196723938
8000,-147793.09257301674,303.87862712692686,12.9,93.82368302345276
9000,-148469.57810036556,303.28420118574775,12.9,107.43151688575745
10000,-148691.72964441185,298.11603308351334,12.9,120.87017703056335


In [16]:
model.addSolvent?